In [15]:
import pandas as pd
import re


In [16]:
def read_tbl():
    with open("../../ocr/data/output/PIB Gist 2015 - COMSTAT 2015 Redacted - Req23-1092 - Rec 4 Dec 2023.txt", "r") as file:
        content = file.read()

    incidents = re.split(r"(\n\d+\. )", content)[1:]  # Split the content into individual incidents

    # Combine the number prefix and incident text
    incidents = [prefix + incident for prefix, incident in zip(incidents[0::2], incidents[1::2])]

    df = pd.DataFrame({"incident": incidents})
    df["incident"] = df["incident"].str.strip()  # Remove leading/trailing whitespace from each incident

    return df

def extract_tracking_id(incident):
    match = re.search(r"\d{4} ?- ?\d{4}[A-Z]", incident)
    if match:
        return match.group()
    else:
        match = re.search(r"\d{4} ?- ?\d{4}", incident)
        return match.group() if match else None

def extract_gist(incident):
    match = re.search(r"(?i)Gist:(.+?)(?=\n\d+\.|$)", incident, re.DOTALL)
    return match.group(1).strip() if match else None

df = read_tbl()
df["tracking_id"] = df["incident"].apply(extract_tracking_id)
df["gist"] = df["incident"].apply(extract_gist)
df.to_csv("../data/output/gist.csv", index=False)